# Imports

In [1]:
import bruce, numpy as np, matplotlib.pyplot as plt, os
from astropy.table import Table
from astropy.stats import sigma_clip

# Load the tics

Data should have the form of (ascii or CSV, your choice). t_zero is in full BJD, width is in days, depth is in normalised flux. If you ue SPOCFIT, if you fit a single transit you will see 3 reported values on the bottom row which are these values for each event (t_zero, width, depth).

tic_id,	t_zero_1,	width_1,	depth_1,	t_zero_2,	width_2,	depth_2,

In [10]:
duos = Table.read('duos.csv')
print(duos)

  tic_id      t_zero_1     width_1     depth_1       t_zero_2     width_2     depth_2  
--------- --------------- ---------- ----------- --------------- ---------- -----------
207501148 2459007.7301472 0.16433394 0.007730663 2459671.8785803 0.13967475 0.007416805


# Now the main worker function

This function does the following:
-  Load the latest TESS data
-  Flatten the lightcurve
-  Fit the events
-  Calcualte the aliases
-  Plot the permissable aliases
-  Create a report

In [ ]:
i=0


# Create the output dir (we'll use this as cache for the data too)
output_dir = os.getcwd() + '/{:}'.format(duos['tic_id'][i])
os.system('mkdir -p {:}'.format(output_dir))

# Now load the TESS data (SPOC, QLP)
# We are not making our own here like TESSTPF, not yet anyway...
# for data_type
#   single_product -> all sectors together
#   per_sector -> list of per-sector lightcurves
#   northern_duos -> YEARS 2 and 4, then a list of other sectors 
#   southern -> YEARS 1 and 3, then a list of other sectors (NOT IMPLEMENTED YET) 
data, data_labels =  bruce.ambiguous_period.get_tess_data(duos['tic_id'][i], 
                                                          download_dir=output_dir, 
                                                          max_sector=np.inf, 
                                                          data_type='per_sector',
                                                         bin_length=0.5/24,
                                                         sigma=None)

# Now flatten the data
for j, k in zip(data, data_labels):
    # Flatten the data by SG filter, we need an odd kernel legth based on cadence
    cadence = np.median(np.gradient(j.time))*1440 # Cadence in minutes
    if cadence > 20 : window_length = 101 # For 30 minute data
    elif cadence > 5 : window_length = 303# for 10 minute data 
    else : window_length = 2003 # for eventual 120-s data 
    sigmaclip=3
    window_length=1001

    # We can cheat like this if we need to tailor a kernel to a specific target
    # if duos['tic_id'][i]==130714841:
    #     window_length = 503

    # Now flatten
    j.w = np.ones(len(j.flux))
    j.flatten_data_old(window_length=window_length, sigmaclip=sigmaclip, dx_lim=0.1)
    
    for seg in bruce.data.find_nights_from_data(j.time, dx_lim=0.2):
        j.w = np.ones(j.time.shape[0])*np.median(j.flux)

    # Optinally save the data
    j.write_data(output_dir + '/' +'TESS_DATA_{:}.txt'.format(k))
    fig, ax = j.plot_segments(dx_lim=0.5)
    fig.savefig(output_dir + '/' + 'TESS_DATA_{:}.png'.format(k))
    plt.close(fig)

# # Now re-order_datasets based on epochs given
# We will unpack now, (data with transits, and data without)
# We may need to change this for it to work properly (Sam is working on it)
# Its worth noting we can incorparate ground based data here too
# data_from_ground = bruce.ambiguous_period.mono_event.photometry_time_series(time, flux, flux_err, w = norm_model)
# Then this can go in data_other_sectors
data, data_labels = bruce.ambiguous_period.group_data_by_epochs(data, data_labels, duos['t_zero_1'][i], duos['t_zero_2'][i])
data, data_other_sectors = data[0], data[1:]



############################
# FIT EVENT 1
############################
# Mask data and create the mono_event object
nmask = 3
mask1 = (data.time > (duos['t_zero_1'][i] - nmask*duos['width_1'][i])) &  (data.time < (duos['t_zero_1'][i] + nmask*duos['width_1'][i]))
data_event_1 = bruce.ambiguous_period.photometry_time_series(data.time[mask1], data.flux[mask1], data.flux_err[mask1], w=data.w[mask1]) #np.percentile(data.flux[mask1], 50)*np.ones(data.time[mask1].shape[0])
m1 = bruce.ambiguous_period.mono_event(duos['t_zero_1'][i], duos['width_1'][i], duos['depth_1'][i], data_event_1, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = 3)

# Fit the event and report plots
fig_initial, ax_initial, fig_final, ax_final, return_data_1 = m1.fit_event_with_fixed_period(fit_period=30., plot=True, )
fig_initial.tight_layout()
fig_final.tight_layout()
fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_1_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
plt.close(fig_initial); plt.close(fig_final)


############################
# FIT EVENT 2
############################
# Mask data and create the mono_event object
mask2 = (data.time > (duos['t_zero_2'][i] - nmask*duos['width_2'][i])) &  (data.time < (duos['t_zero_2'][i] + nmask*duos['width_2'][i]))
data_event_2 = bruce.ambiguous_period.photometry_time_series(data.time[mask2], data.flux[mask2], data.flux_err[mask2], w=data.w[mask2]) #np.percentile(data.flux[mask2], 50)*np.ones(data.time[mask2].shape[0])
m2 = bruce.ambiguous_period.mono_event(duos['t_zero_2'][i], duos['width_2'][i], duos['depth_2'][i], data_event_2, name='TIC-{:}'.format(duos['tic_id'][i]), median_bin_size = None,convolve_bin_size = 3)

# Fit the event and report plots
fig_initial, ax_initial, fig_final, ax_final, return_data_2 = m2.fit_event_with_fixed_period(fit_period=30., plot=True, )
fig_initial.tight_layout()
fig_final.tight_layout()
fig_initial.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_INITIAL.png'.format(duos['tic_id'][i]))
fig_final.savefig(output_dir + '/' + 'TIC-{:}_EVENT_2_INITIAL_FINAL.png'.format(duos['tic_id'][i]))
plt.close(fig_initial); plt.close(fig_final)

# We are going to make a nice plot of the two events with their models
fig, ax = plt.subplots(1,2, gridspec_kw={'hspace' : 0, 'wspace' : 0}, figsize = (6.4, 3.8))
ax[0].errorbar(return_data_1[0], return_data_1[1], yerr=return_data_1[2], fmt='k.', alpha = 0.1)
ax[0].plot(return_data_1[3], return_data_1[4], c='orange')
ax[1].errorbar(return_data_2[0], return_data_2[1], yerr=return_data_2[2], fmt='k.', alpha = 0.1)
ax[1].plot(return_data_2[3], return_data_2[4], c='orange')
ax[1].set(yticks=[])
ylim1 = ax[0].get_ylim()
ylim2 = ax[1].get_ylim()
ylim = [min(ylim1[0],ylim2[0]), max(ylim1[1], ylim2[1])]
ax[0].set_ylim(ylim)
ax[1].set_ylim(ylim)
fig.supxlabel('Time from Transit [d]', fontsize=18, x=0.55, y = -0.005)
fig.supylabel('Flux', fontsize=18)
fig.suptitle(m2.name, y=0.95, x=0.55, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3', alpha=1.0), ha='center', fontsize=18)
plt.subplots_adjust(right=0.99, top=0.99, bottom=0.13)
fig.savefig(output_dir + '/' + 'TIC-{:}_BOTH_EVENTS.png'.format(duos['tic_id'][i]))
plt.close(fig)

########################################################
# CREATE THE AMBIGUOUS PERIOD OBJECT
########################################################
p = bruce.ambiguous_period.ambiguous_period(data, events=[m1,m2], name='TIC-{:}'.format(duos['tic_id'][i]),
                    median_bin_size = 2,convolve_bin_size = 2)

# Now mask and filter 
p.mask_and_filter_events()

# Calculate aliases
# Do not use nsolutions_events here (that is superceeded later)
nsolutions_events = p.calcualte_aliases(dx_lim=0.03, min_period=15)

# Now calcualte whether we saw a transit by comparing the model to a flat line
delta_L_data = p.calcualte_data_delta_L(data)
#delta_L_data = delta_L_data + 100 # THIS FUDGE IS OFTEN NEEDED
for j in range(delta_L_data.shape[0]):
    print(j, delta_L_data[j])
delta_L_data_from_other_sectors_or_others = [p.calcualte_data_delta_L(j) for j in data_other_sectors]
p.delta_L = np.array([delta_L_data, *delta_L_data_from_other_sectors_or_others])


########################################################
# Plot the aliases
########################################################
fig, ax  = p.plot_aliases(phot_data=data_other_sectors, phot_data_labels=data_labels)
fig.savefig(output_dir + '/' + 'TIC-{:}_ALIASES.png'.format(duos['tic_id'][i]), dpi=300)
plt.close(fig)

        
        

INFO: Found cached file /Users/sam/Software/bruce/examples/duotransits/207501148/mastDownload/HLSP/hlsp_tess-spoc_tess_phot_0000000207501148-s0016_tess_v1_tp/hlsp_tess-spoc_tess_phot_0000000207501148-s0016_tess_v1_lc.fits with expected size 149760. [astroquery.query]


INFO: Found cached file /Users/sam/Software/bruce/examples/duotransits/207501148/mastDownload/HLSP/hlsp_tess-spoc_tess_phot_0000000207501148-s0019_tess_v1_tp/hlsp_tess-spoc_tess_phot_0000000207501148-s0019_tess_v1_lc.fits with expected size 149760. [astroquery.query]


INFO: Found cached file /Users/sam/Software/bruce/examples/duotransits/207501148/mastDownload/HLSP/hlsp_tess-spoc_tess_phot_0000000207501148-s0022_tess_v1_tp/hlsp_tess-spoc_tess_phot_0000000207501148-s0022_tess_v1_lc.fits with expected size 161280. [astroquery.query]


INFO: Found cached file /Users/sam/Software/bruce/examples/duotransits/207501148/mastDownload/HLSP/hlsp_tess-spoc_tess_phot_0000000207501148-s0023_tess_v1_tp/hlsp_tess-spoc_tess_phot_0000000207501148-s0023_tess_v1_lc.fits with expected size 158400. [astroquery.query]


INFO: Found cached file /Users/sam/Software/bruce/examples/duotransits/207501148/mastDownload/HLSP/hlsp_tess-spoc_tess_phot_0000000207501148-s0024_tess_v1_tp/hlsp_tess-spoc_tess_phot_0000000207501148-s0024_tess_v1_lc.fits with expected size 158400. [astroquery.query]


...

INFO: Found cached file /Users/sam/Software/bruce/examples/duotransits/207501148/mastDownload/TESS/tess2024142205832-s0079-0000000207501148-0274-s/tess2024142205832-s0079-0000000207501148-0274-s_lc.fits with expected size 1984320. [astroquery.query]


INFO: Found cached file /Users/sam/Software/bruce/examples/duotransits/207501148/mastDownload/TESS/tess2024249191853-s0083-0000000207501148-0280-s/tess2024249191853-s0083-0000000207501148-0280-s_lc.fits with expected size 1825920. [astroquery.query]


INFO: Found cached file /Users/sam/Software/bruce/examples/duotransits/207501148/mastDownload/TESS/tess2024326142117-s0086-0000000207501148-0283-s/tess2024326142117-s0086-0000000207501148-0283-s_lc.fits with expected size 1944000. [astroquery.query]
W :  1001


W :  1001
W :  1000
W :  1000
W :  1000
W :  1000
W :  1000
W :  1000
W :  400
W :  1000
W :  1000
W :  1001
W :  1000
W :  1001
W :  1001
W :  1001
W :  1001
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1001
W :  1001
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  1002
W :  250
W :  501
W :  1002
W :  1002
W :  1002
W :  1002
Initial Chi-Sqaured : 1031.28 [red 23.98]
Fitted parameters for TIC-207501148:
t_zero : 2459007.728495915
radius_1 : 0.031075756033523277
k : 0.08903077066142523
b : 0.8994095156743517
Final Chi-Sqaured : 156.03 [red 3.63]
Initial Chi-Sqaured : 3151.75 [red 95.51]
Fitted parameters for TIC-207501148:
t_zero : 2459671.88671243
radius_1 : 0.015983562056

100%|█████████████████████████████████████████████████████████| 44/44 [00:39<00:00,  1.12it/s]


In [4]:
data[5].time

TypeError: 'photometry_time_series' object is not subscriptable

In [ ]:
data_labels

In [ ]:
data, data_labels =  bruce.ambiguous_period.get_tess_data(duos['tic_id'][i], 
                                                          download_dir=output_dir, 
                                                          max_sector=np.inf, 
                                                          data_type='per_sector',
                                                         bin_length=0.5/24,
                                                         sigma=3)

In [ ]:
d = data[0]


In [ ]:
data[1].time